In [1]:
# Fits a polynomial over the peak between start_range and end_range. Uses the local maxima of the second 
# derivative of that polynomial to determine the cutoff points for the peak itself. Fits a polynomial to 
# the data that excludes the peak, and subtracts that background polynomial from the spectrum data to 
# give final peak

# lorentzian fit to peak to give heights

def bkg_fit(xdata, ydata, center, span, sum_fitted_ydata, sum_fixed_ydata):
    
# make data an array
    x_np = np.array(xdata)
    y_np = np.array(ydata)

    # select peak
    # argmax finds index of start/end of peak range based on user inputted peak range
    # x_peak = x_np[(np.argmax(x_np>start_peak_range)):(np.argmax(x_np>end_peak_range))]
    # y_peak = y_np[(np.argmax(x_np>start_peak_range)):(np.argmax(x_np>end_peak_range))]
    
    #span = round(0.2*len(x_np))
    span_idx1 = np.argmax(np.where(x_np<center-span))
    span_idx2 = np.argmax(np.where(x_np<center+span))
    x_peak = x_np[span_idx1:span_idx2]
    y_peak = y_np[span_idx1:span_idx2]
    start_peak = center-45
    
    # POLYNOMIAL FIT TO SPECTRUM

    # degree of polynomial fit
    degree = 10

    # calculate polynomial
    # polyfit fits x and y data to a polynomoinal of given degrees. poly1d tells Python the values are a polynomial
    z = np.polyfit(x_peak, y_peak, degree)
    f = np.poly1d(z)

    # calculate y values for polynomial
    y_poly = f(x_peak)

    # derivative
    import scipy.misc as sm
    derivs2 = []
    
    for i in x_peak:
        derivs2.append((sm.derivative(f, i, n = 2)))

    # FIND LOCAL MAXIMA OF SECOND DERIVATIVE

    # degree of polynomial fit
    degree2 = 10

    # calculate polynomial
    z2 = np.polyfit(x_peak, derivs2, degree2)
    f2 = np.poly1d(z2)

    # calculate y values for polynomial
    y2 = f2(x_peak)

    # find local maxima 
    from scipy.signal import argrelextrema

    # use start_peak to eliminate edge effects
    # argmax find the index of the largest x value after the user inputted peak start and uses this to isolate the precise
    # range of x data for the peak
    # idx = np.argmax(x_peak>start_peak)
    # derivs2a = np.array(y2[idx:])

    # this is indexed from the start of x_peak
    # finds the indices of the relative maxes in 2nd derivative of each peak then isolates start and end indices. Yields final
    # start and end indices of each peak
    maxs = argrelextrema(y2, np.greater)
    if len(maxs[0][:]) == 3:
        maxs = maxs[0][1:]
        start_idx = maxs[0]
        end_idx = maxs[1]
    elif len(maxs[0][:]) > 3:
        maxs = maxs[0][1:-1]
        start_idx = maxs[0]
        end_idx = maxs[1]
    elif len(maxs[0][:]) < 3:
        start_idx = maxs[0][0]
        end_idx = maxs[0][1]
        
    fixed_start_peak = x_peak[start_idx]
    fixed_end_peak = x_peak[end_idx]

    # FIT CURVE TO BACKGROUND 

    # exclude peak
    # uses start and end indices of each peak to remove peak and isolate background
    x_no_peak = np.concatenate((x_peak[:start_idx], x_peak[end_idx:]))
    y_no_peak = np.concatenate((y_peak[:start_idx], y_peak[end_idx:]))

    # degree of polynomial fit to background
    degree3 = 3

    # calculate polynomial
    z3 = np.polyfit(x_no_peak, y_no_peak, degree3)
    f3 = np.poly1d(z3)

    # SUBTRACT OUT BACKGROUND
    # fixed as in fitted background was removed
    y_peak_fixed = y_peak - f3(x_peak)
    
    # CURVE FITTING
    from scipy.optimize import curve_fit
    from math import pi
    
    # rename x and y peak variables
    xdata_peak = x_peak
    ydata_peak = y_peak_fixed

    # initial parameters (guess - doesn't really matter), load into an array
    init_center = center
    init_width = 100
    init_amplitude_factor = 10

    init_params = [init_center, init_width, init_amplitude_factor]

    # lorentzian peak function
    def lorentz(x, center, width, A):
        return ( A * (0.5 * width) / (pi * ( ((x - center) ** 2) + ((0.5 * width) ** 2) ) ))
    
    #pass function, x and y peak data, and initial params to curve fitting module. Generate optimized params
    params, cov = curve_fit(lorentz, xdata_peak, ydata_peak, p0=init_params)
    #calculate amplitude (height), FWHM, and peak center using params
    height = params[2] * 2 / (pi * params[1])
    FWHM = params[1]
    center = params[0]
    #load fitted y data and fixed y data into arrays in correct index positions to be passed back to back_sub_curve_fitting
    sum_fitted_ydata[span_idx1:span_idx2] = lorentz(xdata_peak, *params)
    sum_fixed_ydata[span_idx1:span_idx2] = ydata_peak
    
    return(height, FWHM, center, sum_fitted_ydata, sum_fixed_ydata)

In [2]:
def bkg_fit_with_plots(xdata, ydata, center, span, sum_fitted_ydata, sum_fixed_ydata):
    
    # make data an array
    x_np = np.array(xdata)
    y_np = np.array(ydata)

    # select peak
    # argmax finds index of start/end of peak range based on user inputted peak range
    # x_peak = x_np[(np.argmax(x_np>start_peak_range)):(np.argmax(x_np>end_peak_range))]
    # y_peak = y_np[(np.argmax(x_np>start_peak_range)):(np.argmax(x_np>end_peak_range))]
    
    #span = round(0.35*len(x_np))
    span_idx1 = np.argmax(np.where(x_np<center-span))
    span_idx2 = np.argmax(np.where(x_np<center+span))
    x_peak = x_np[span_idx1:span_idx2]
    y_peak = y_np[span_idx1:span_idx2]
    start_peak = center-45
    
    # POLYNOMIAL FIT TO SPECTRUM

    # degree of polynomial fit
    degree = 10

    # calculate polynomial
    # polyfit fits x and y data to a polynomoinal of given degrees. poly1d tells Python the values are a polynomial
    z = np.polyfit(x_peak, y_peak, degree)
    f = np.poly1d(z)

    # calculate y values for polynomial
    y_poly = f(x_peak)

    # derivative
    import scipy.misc as sm
    derivs2 = []
    
    for i in x_peak:
        derivs2.append((sm.derivative(f, i, n = 2)))

    # FIND LOCAL MAXIMA OF SECOND DERIVATIVE

    # degree of polynomial fit
    degree2 = 10

    # calculate polynomial
    z2 = np.polyfit(x_peak, derivs2, degree2)
    f2 = np.poly1d(z2)

    # calculate y values for polynomial
    y2 = f2(x_peak)

    # find local maxima 
    from scipy.signal import argrelextrema

    # use start_peak to eliminate edge effects
    # argmax find the index of the largest x value after the user inputted peak start and uses this to isolate the precise
    # range of x data for the peak
    # idx = np.argmax(x_peak>start_peak)
    # derivs2a = np.array(y2[idx:])

    # this is indexed from the start of x_peak
    # finds the indices of the relative maxes in 2nd derivative of each peak then isolates start and end indices. Yields final
    # start and end indices of each peak
    maxs = argrelextrema(y2, np.greater)
    if len(maxs[0][:]) == 3:
        maxs = maxs[0][1:]
        start_idx = maxs[0]
        end_idx = maxs[1]
    elif len(maxs[0][:]) > 3:
        maxs = maxs[0][1:-1]
        start_idx = maxs[0]
        end_idx = maxs[1]
    elif len(maxs[0][:]) < 3:
        start_idx = maxs[0][0]
        end_idx = maxs[0][1]
        
    fixed_start_peak = x_peak[start_idx]
    fixed_end_peak = x_peak[end_idx]

    # FIT CURVE TO BACKGROUND 

    # exclude peak
    # uses start and end indices of each peak to remove peak and isolate background
    x_no_peak = np.concatenate((x_peak[:start_idx], x_peak[end_idx:]))
    y_no_peak = np.concatenate((y_peak[:start_idx], y_peak[end_idx:]))

    # degree of polynomial fit to background
    degree3 = 3

    # calculate polynomial
    z3 = np.polyfit(x_no_peak, y_no_peak, degree3)
    f3 = np.poly1d(z3)

    # SUBTRACT OUT BACKGROUND
    # fixed as in fitted background was removed
    y_peak_fixed = y_peak - f3(x_peak)
    
    # CURVE FITTING
    from scipy.optimize import curve_fit
    from math import pi
    
    # rename x and y peak variables
    xdata_peak = x_peak
    ydata_peak = y_peak_fixed

    # initial parameters (guess - doesn't really matter), load into an array
    init_center = center
    init_width = 100
    init_amplitude_factor = 10

    init_params = [init_center, init_width, init_amplitude_factor]

    # lorentzian peak function
    def lorentz(x, center, width, A):
        return ( A * (0.5 * width) / (pi * ( ((x - center) ** 2) + ((0.5 * width) ** 2) ) ))
    
    #pass function, x and y peak data, and initial params to curve fitting module. Generate optimized params
    params, cov = curve_fit(lorentz, xdata_peak, ydata_peak, p0=init_params)
    #calculate amplitude (height), FWHM, and peak center using params
    height = params[2] * 2 / (pi * params[1])
    FWHM = params[1]
    center = params[0]
    #load fitted y data and fixed y data into arrays in correct index positions to be passed back to back_sub_curve_fitting
    sum_fitted_ydata[span_idx1:span_idx2] = lorentz(xdata_peak, *params)
    sum_fixed_ydata[span_idx1:span_idx2] = ydata_peak
    
    # PLOTS 

    plt.figure(0)
    plt.title('Spectrum')
    plt.scatter(x_np, y_np)
    plt.show()

    plt.figure(1)
    plt.title('Polynomial Fit')
    plt.scatter(x_peak,y_peak)
    plt.plot(x_peak,y_poly,'-r')
    plt.show()

    plt.figure(2)
    plt.title('Polynomial Fit of Second Derivative')
    plt.scatter(x_peak,derivs2)
    plt.plot(x_peak, y2,'-r')
    plt.show()

    plt.figure(3)
    plt.title('Spectrum with Cutoffs')
    plt.axvline(x=fixed_start_peak, linestyle=':', color='r')
    plt.axvline(x=fixed_end_peak, linestyle=':', color='r')
    plt.scatter(x_peak,y_peak)
    plt.show()

    plt.figure(4)
    plt.title('Curve Fit No Peak')
    plt.scatter(x_peak, y_peak)
    plt.plot(x_peak, f3(x_peak), linestyle=':', color='r')
    plt.show()

    plt.figure(5)
    plt.title('Background Subtracted')
    plt.scatter(x_peak, y_peak_fixed)
    plt.show()

    
    plt.figure(6)
    plt.title('Final Fit')
    plt.scatter(xdata_peak, ydata_peak)
    plt.plot(xdata_peak, lorentz(x_peak, *params), color='r')
    plt.show() 
    
    return(height, FWHM, center, sum_fitted_ydata, sum_fixed_ydata)

NameError: name 'np' is not defined